In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


### 1단계

In [88]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

In [101]:
# Load Data
loader = TextLoader(file_path="data/taxinfo.txt", encoding="utf-8")
documents = loader.load()
print(type(documents[0]))
print(type(documents))
print(documents)
print(documents[0].page_content[-20:-1])
print(documents[0].metadata)
print(documents[0].metadata["source"])

<class 'langchain_core.documents.base.Document'>
<class 'list'>
[Document(metadata={'source': 'data/taxinfo.txt'}, page_content='제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31.>\n1. 「공익신탁법」에 따른 공익신탁의 이익\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n    가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n    나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n    다. 대통령령으로 정하는 농어가부업소득\n    라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n    마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n    바. 대통령령

In [102]:
# 2️. Text Split
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(documents)
print(len(split_docs))
print(split_docs[11].page_content)
# for doc in split_docs:
#     print(doc.page_content)

12
아. 제21조제1항제26호에 따른 종교인소득 중 다음의 어느 하나에 해당하는 소득
    　　　　1) 「통계법」 제22조에 따라 통계청장이 고시하는 한국표준직업분류에 따른 종교관련종사자(이하 “종교관련종사자”라 한다)가 받는 대통령령으로 정하는 학자금
    　　　　2) 종교관련종사자가 받는 대통령령으로 정하는 식사 또는 식사대
    　　　　3) 종교관련종사자가 받는 대통령령으로 정하는 실비변상적 성질의 지급액
    　　　　4) 종교관련종사자 또는 그 배우자의 출산이나 6세 이하(해당 과세기간 개시일을 기준으로 판단한다) 자녀의 보육과 관련하여 종교단체로부터 받는 금액으로서 월 20만원 이내의 금액
    　　　　5) 종교관련종사자가 기획재정부령으로 정하는 사택을 제공받아 얻는 이익
    자. 법령ㆍ조례에 따른 위원회 등의 보수를 받지 아니하는 위원(학술원 및 예술원의 회원을 포함한다) 등이 받는 수당


In [ ]:
# 3️. Indexing (벡터 저장) - RAG(Retrieval-Augmented Generation) 시스템의 핵심
"""
# FAISS(Facebook AI Similarity Search): 페이스북 AI에서 개발한 오픈소스 라이브러리
    # 목적: 
        # 대규모 벡터 집합에서 효율적으로 유사성 검색을 수행하는 것
        # 즉 대규모의 숫자 벡터(임베딩) 중 특정 쿼리 벡터와 가장 가까운(유사한) 벡터들을 빠르게 찾아내는 기술
    # Embedding: 컴퓨터가 이해할 수 있도록 비정형 데이터(텍스트, 이미지, 소리 등)의 의미를 숫자의 배열(임베딩 벡터)로 변환하는 과정
# .from_documents(): LangChain에서 FAISS를 사용하여 백터 저장소를 생성하는 클래스 메서드 
    # 입력: 
        # documents:  
            # 백터화 할 Document 객체들의 리스트. 
            # 각 Document 객체는 page_content(실제 텍스트 내용)와 metadata(관련정보)를 포함
        # embedding:
            # 텍스트를 벡터로 변환할 임베딩 모델 객체
    # 수행하는 작업 단계: 
        # 1. documents 순회: 
                    # documents 리스트에 있는 Document 객체를 하나씩 가져옴
        # 2. 임베딩 생성: 
                    # 각 Document 객체의 page_content(텍스트 내용)를 embedding 모델에 전달.
                    # embedding 모델은 해당 텍스트에 해당하는 고차원 숫자 벡터(임베딩)를 생성
        # 3. FAISS 인덱스 구축: 
                    # 생성 된 임베딩 벡터와 해당 벡터에 연결된 원본 텍스트(page_content) 및 메타 데이터를 FAISS 인덱스 내부에 저장. 
                    # FAISS는 해당 벡터(임베딩)들을 효율적으로 검색할 수 있도록 내부적으로 색인화하고 구조화 함
                        # (색인화: 데이터를 나중에 빠르고 효율적으로 찾을 수 있도록 미리 정리하고 특정 구조로 저장하는 과정)
        # 4. FAISS 벡터 저장소 객체 반환: 
                    # 모든 과정이 완료되면 FAISS.from_documents()는 FAISS 클래스의 인스턴스(객체)를 반환. 
                    # 해당 객체는 테스트 덩어리들의 임베딩과 해당 텍스트를 모두 포함하고 있으며 나중에 유사성 검색을 수행할 준바가 되어 있음. 
    # 0. 요약: 
        # documents 리스트에 있는 모든 텍스트 덩어리들을 가져와서, 
        # embedding에 저장 된 모델을 통해 의미를 담은 고차원 숫자 벡터(임베딩 백터)로 변환하고, 
        # 해당 벡터들을 FAISS라는 고속 검색 엔진에 효율적으로 저장(인덱싱/색인화)하여 
        # FAISS 클래스의 인스턴스(검색 가능한 데이터베이스 객체)를 반환. 
"""
vectorstore = FAISS.from_documents(documents=split_docs, embedding=OpenAIEmbeddings()) # 우선 변수로 메모리에 저장한 후

# 로컬 파일로 저장 
# 메모리 상에 구축 된 벡터 저장소(FAISS 인덱스)를 영구적으로 로컬 컴퓨터의 디스크에 저장하는 역할을 함
vectorstore.save_local("faiss_index") # faiss_index라는 이름으로 로컬 파일로 저장
print(type(vectorstore))
print(vectorstore)

<class 'langchain_community.vectorstores.faiss.FAISS'>


In [104]:
# 4️. Retrieval (유사 문서 검색)
"""
# .as_retriever():
    # 이전에 구축하고 저장했던 FAISS 인덱스 객체로 부터 Retriever 겍체 생성
    # Retriever는 LangChain에서 주어진 쿼리에 대해 문서를 검색하는 추상화 된 인터페이스
        # 즉 어떤 방식으로 검색하던 상관 없이 Retriever는 항상 invoke()메서드를 사용하여 쿼리를 입력받고 Document 객체 리스트를 반환하는 표준화 된 방식 제공
    # 입력:
        # search_kwargs: Retriever가 문서를 검색할 때 사용할 추가적인 검색 파라미터 설정하는 속성
            # {"k": 4}: 가장 유사한 문서 4개를 검색하라는 의미. 
                # 즉 사용자의 질문과 벡터 공간에서 가장 가까운 텍스트 덩어리 4개를 찾아오라고 지시하는 것
"""
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# **질문(쿼리)**에 대해 유사한 문서를 검색하는 역할
"""
# .invoke( ... ): Retriever 객체의 핵심 메서드. 여기에 실제 사용자 질문(퀴리) 문자열을 전달
    # 내부 동작:
        # 1. ... 라는 질문 텍스트를 vectorstore에 저장 된 FAISS 인스턴스의 인자값 embedding에 저장되어 있는 OpenAIEmbeddings() 모델을 사용하여 임베딩 벡터로 변환
        # 2. 변환 된 질문 임베딩 벡터를 FAISS 인덱스에 전달하여 유사성 검색 수행
        # 3. search_kwargs={"k": 4}에 따라 FAISS는 질문 임베딩과 가장 유사한(가장 가까운) 상위 4개의 Document 객체를 찾아 반환
        # 4. 이렇게 검색 된 4개의 Document 객체들이 리스트 형태로 retrieved_docs 변수에 할당됨
"""
retrieved_docs = retriever.invoke("소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
print(type(retrieved_docs))
pprint(retrieved_docs)


<class 'list'>
[Document(id='281cfc57-bcdc-43d9-8a41-b689c421c277', metadata={'source': 'data/taxinfo.txt'}, page_content='제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31.>\n1. 「공익신탁법」에 따른 공익신탁의 이익\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n    가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득'),
 Document(id='4415ff59-fbf9-42ba-b839-48e98eaab99e', metadata={'source': 'data/taxinfo.txt'}, page_content='가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n    나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n    다. 대통령령으로 정하는 농어가부업소득\n    라. 대통령령으로 정하는 전통주의 제

In [105]:
# 5️. Generation (LLM 응답 생성)
llm = ChatOpenAI(model="gpt-4o")
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
print(context)

제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31.>
1. 「공익신탁법」에 따른 공익신탁의 이익
2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득
    가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득

가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득
    나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.
    다. 대통령령으로 정하는 농어가부업소득
    라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득
    마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.
    바. 대통령령으로 정하는 작물재배업에서 발생하는 소득

마. 「고용보험법」에 따라 받는 실업급여, 육아휴직 급여, 육아기 근로시간 단축 급여, 출산전후휴가 급여등, 「제대군인 지원에 관한 법률」에 

In [106]:
response_context = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}")
print('context 적용한 결과')
pprint(response_context.content)

context 적용한 결과
('소득세법 제12조에 따라 비과세 소득으로 인정되는 소득 유형은 다음과 같습니다:\n'
 '\n'
 '1. **공익신탁의 이익**: 「공익신탁법」에 따른 공익신탁에서 발생하는 이익은 소득세가 부과되지 않습니다.\n'
 '\n'
 '2. **일부 사업소득**:\n'
 '   - 작물 생산을 목적으로 논과 밭을 이용하여 발생하는 소득\n'
 '   - 1개의 주택을 소유하는 자의 주택 임대 소득 (일정 조건 하에서)\n'
 '   - 대통령령으로 정하는 농어가부업소득\n'
 '   - 전통주의 제조에서 발생하는 소득\n'
 '   - 5년 이상의 조림지에서 발생하는 소득 중 연 600만 원 이하의 소득\n'
 '   - 대통령령으로 정하는 작물재배업의 소득\n'
 '   - 대통령령으로 정하는 어로어업 및 양식어업의 소득\n'
 '\n'
 '3. **고용 및 복지 관련 소득**:\n'
 '   - 「고용보험법」에 따른 실업급여, 육아휴직 급여, 출산전후휴가 급여 등\n'
 '   - 「제대군인 지원에 관한 법률」에 따른 전직지원금\n'
 '   - 공무원이나 사립학교 교직원이 받는 육아휴직수당 (일정 조건 하에서)\n'
 '   - 「국민연금법」에 따른 특정 반환일시금 및 사망일시금\n'
 '\n'
 '4. **근로 및 퇴직 소득**:\n'
 '   - 병(兵) 복무 중에 받는 급여 (대통령령에 따라 규정)\n'
 '   - 법률에 따라 동원된 직장에서 받는 급여\n'
 '   - 「산업재해보상보험법」에 따라 받는 다양한 보상금\n'
 '   - 「근로기준법」 또는 「선원법」에 따라 받는 요양보상금, 휴업보상금 등\n'
 '\n'
 '이러한 비과세 소득은 법령에 따라 정확히 적용되는 조건과 범위가 정해져 있으며, 특정한 경우에 한해 소득세 부과를 면제받습니다.')


In [107]:
response = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
print('context 적용하지 않은 결과')
pprint(response.content)

context 적용하지 않은 결과
('소득세법에서 비과세 소득은 과세 대상에서 제외되는 소득을 의미합니다. 이는 정부가 특정한 소득에 대해 세금을 부과하지 않기로 결정한 '
 '경우에 해당합니다. 한국의 소득세법에 따라 비과세로 분류되는 소득은 다음과 같은 것들이 포함될 수 있습니다.\n'
 '\n'
 '1. **이자소득**: 일정 조건을 충족하는 장기주택저축이나 생계형 저축에서 발생하는 이자소득이 비과세될 수 있습니다.\n'
 '\n'
 '2. **배당소득**: 일정 조건을 충족하는 경우 비과세되는 배당소득이 있을 수 있습니다.\n'
 '\n'
 '3. **근로소득**: 특정 조건 하의 야간근로나 휴일 근로수당에 대한 비과세 혜택이 주어질 수 있습니다.\n'
 '\n'
 '4. **연금소득**: 국민연금 수령액의 일부는 비과세될 수 있습니다.\n'
 '\n'
 '5. **기타**: 상속이나 증여로 인한 소득의 일부, 또는 사망보험금 같은 소득도 비과세 대상이 될 수 있습니다.\n'
 '\n'
 '비과세 소득의 정확한 조건과 범위는 관련 법률과 세법 개정에 따라 달라질 수 있기 때문에, 최신 정보를 제공받기 위해서는 국세청이나 관련 '
 '법률 전문가와 상담하는 것이 좋습니다.')


### 2단계

In [109]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. 데이터 로드 (기존과 동일)
loader = TextLoader(file_path="data/taxinfo.txt", encoding="utf-8")
documents = loader.load()



# 2. 텍스트 분할 개선
splitter = RecursiveCharacterTextSplitter(  # 텍스트 분할기
    chunk_size=1000,        # 크기 증가 최대 1000자까지
    chunk_overlap=200,      # 동일 단어 200자까지 가능
    separators=["\n\n", "\n", " ", ""],     # 자연스러운 분할을 위한 구분자
    length_function=len,        # chunk_size와 chunk_overlap을 계산할 때 텍스트의 길이를 측정하는데 사용할 메서드 len()
    is_separator_regex=False,   # separators 리스트에 정의 된 구분자들이 정규 표현식으로 해석될 지 여부를 결정. False이기에 "\n\n", "\n", " ", ""는 일반 문자열 구분자로 사용됨
)
split_docs = splitter.split_documents(documents=documents) # 텍스트 분할기를 사용해서 documents를 나눔



# 3. 인덱싱 (벡터 저장)
# split_docs를 컴퓨터가 읽을 수 있는 임베딩 벡터로 변환한 뒤 FAISS라는 고속 검색 엔진 클래스 객체로 vectorstore 변수에 저장(메모리에만)
vectorstore = FAISS.from_documents(documents=split_docs, embedding=OpenAIEmbeddings()) 
vectorstore.save_local("faiss_index") # faiss_index라는 이름으로 로컬 파일로 저장


# 4. 검색 개선
retriever = vectorstore.as_retriever(
    search_type="mmr",  # MMR(Maximal Marginal Relevance): 최대 다양성 검색
    search_kwargs={     # 더 많은 결과 검색
        # mmr 알고리즘을 통해 선택 될 문서의 갯수. retriever.invoke( ... )가 반환할 Document 인스턴스의 갯수 5개
        "k": 5,         
        # mmr 알고리즘이 다양성을 고려하기 전 먼제 벡터 저장소에서 가져올 초기 유사 문서의 갯수. 
        # mmr은 후보군이 필요하기에 후보군 10개를 가져와서 5개를 선택할
        # fetch_k >= k
        "fetch_k": 10   
        }
)

# 5. 프롬프트 엔지니어링
def generate_prompt(query, context):
    return f"""
            다음은 소득세법 비과세소득 관련 조항입니다. 문맥을 고려하여 질문에 답변하세요.

            [관련 조항]
            {context}

            [질문]
            {query}

            [답변 요구사항]
            - 비과세소득 유형을 계층적으로 구분하여 설명
            - 각 항목별 구체적인 조건 명시
            - 법조문의 항, 호, 목 번호를 포함
            - 500자 이내로 간결하게 요약
            """

# 검색 및 응답 생성
query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

llm = ChatOpenAI(model="gpt-4o", temperature=0.3)  # 창의성 낮춤
response = llm.invoke(generate_prompt(query, context))

print('개선된 결과:')
print(response.content)

개선된 결과:
소득세법 제12조에 따르면 비과세소득은 다음과 같이 구분됩니다:

1. **공익신탁 소득**: 
   - 1항: 「공익신탁법」에 따른 공익신탁의 이익.

2. **사업소득**:
   - 2항: 논밭 임대 소득, 1주택 임대소득(기준시가 12억원 이하), 농어가부업소득, 전통주 제조 소득, 조림기간 5년 이상 임지의 벌채 소득(연 600만원 이하), 작물재배업 소득, 어로어업 소득.

3. **근로소득 및 퇴직소득**:
   - 3항: 국외 근로 급여, 보험료(국가 부담), 특정 근로자의 연장근로 급여, 사내급식, 출산/보육 급여(월 20만원 이하), 국군포로 보수, 대학생 근로 장학금, 직무발명 보상금.

4. **기타소득**:
   - 5항: 보훈급여금, 국가보안법 상금, 훈장 부상, 직무발명 보상금, 국군포로 위로금, 문화유산 양도 소득, 종교인 소득(학자금, 식사, 보육비), 위원회 수당.

각 항목은 대통령령에 의해 구체적인 조건이 정해집니다.


### 3단계

In [112]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. Load Data
loader = TextLoader(file_path="data/taxinfo.txt", encoding="utf-8") # 우선 TextLoader 객체로 객체를 만들어서 loader 변수에 할당
documents = loader.load()   # 이후 Document 객체 또는 Document 객체 리스트로 변환하고 documents에 반환

print("=== 원본 문서 길이 ===")
print(f"전체 문서 길이: {len(documents[0].page_content)} 글자")


# 2. Text Split 개선
# 우선 텍스트 분할기를 만든 뒤
splitter = RecursiveCharacterTextSplitter( # 텍스트 분할기 생성
    chunk_size=800,     # 최대 길이: 500 → 800 (법령 조항이 길어서)
    chunk_overlap=150,  # 최대 중복 단어: 50 → 150 (맥락 보존 강화)
    separators=["\n\n", "\n", ". ", " ", ""]  # 법령 구조에 맞는 분리자
)
# 분할기를 documents 문서에 적용 후 split_docs에 할당
split_docs = splitter.split_documents(documents=documents)

print(f"분할된 문서 수: {len(split_docs)}개")
print("=== 분할 예시 ===")
for i, doc in enumerate(split_docs[:3]):
    print(f"Chunk {i+1} ({len(doc.page_content)}글자): {doc.page_content[:100]}...")


# 3. Indexing
# 우선 분할 된 Document 객체 리스트인 split_docs를 OpenAIEmbeddings() 모델로 컴퓨터가 읽을 수 있도록
# 고차원의 숫자 배열(임베딩 백터)로 변환 후 FAISS라는 검색 속도 향상 클래스로 색인화(더욱 빠르게 찾을 수 있도록 구조화) 후 vectorstore라는 변수의 메모리에 할당
vectorstore = FAISS.from_documents(documents=split_docs, embedding=OpenAIEmbeddings())
# vectorstore라는 변수에 할당 된 고차원의 숫자 배열(임베딩 백터)을 ./db/faiss_index라는 이름으로 로컬에 저장
vectorstore.save_local("./db/faiss_index")


# 4. Retrieval 개선
retriever = vectorstore.as_retriever(
    # search_type 매개변수에 "similarity"를 저장하면 유사성 검색 방식을 사용하라고 지시
    search_type="similarity",
    # serch_kwargs 매개변수에 retriever.invoke( ... )를 진행했을 때 6개까지 Document 인스턴스 객체를 반환하라고 지시
    search_kwargs={"k": 6}
)

query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?" # 질문
retrieved_docs = retriever.invoke(query) # 반환 된 6개의 Document 객체들을 retrieved_docs라는 리스트로 저장

print(f"\n=== 검색된 문서 ({len(retrieved_docs)}개) ===")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}: {doc.page_content[:200]}...")
    print("---")


# 5. Generation - 개선된 프롬프트
llm = ChatOpenAI(model="gpt-4o", temperature=0) # gpt-4o 모델을 temperature 매개변수에 0을 지정해서 창의성을 없앤 모델을 llm에 반환
context = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])


# 개선된 프롬프트 - 더 구체적인 지시사항
improved_prompt = f"""
당신은 세무 전문가입니다. 아래 소득세법 제12조 조항을 바탕으로 질문에 답변해주세요.

질문: {query}

법령 조항:
{context}

다음 형식으로 답변해주세요:
1. 비과세소득의 정의
2. 주요 비과세소득 항목들을 다음과 같이 분류:
   - 사업소득 관련
   - 근로소득/퇴직소득 관련  
   - 연금소득 관련
   - 기타소득 관련
3. 각 항목별 구체적인 조건이나 한도액 명시

답변은 법조문을 인용하면서 구체적으로 작성해주세요.
"""

# 비교용 - 기존 방식
simple_prompt = f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}"

print("=== 기존 프롬프트로 답변 ===")
response_simple = llm.invoke(simple_prompt)
pprint(response_simple.content)

print("\n" + "="*50)
print("\n=== 개선된 프롬프트로 답변 ===")
response_improved = llm.invoke(improved_prompt)
pprint(response_improved.content)

# 추가 개선: 다른 검색 방식 시도
print("\n" + "="*50)
print("=== 검색 방식 개선 테스트 ===")

=== 원본 문서 길이 ===
전체 문서 길이: 4971 글자
분할된 문서 수: 8개
=== 분할 예시 ===
Chunk 1 (738글자): 제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2....
Chunk 2 (636글자): 다. 대통령령으로 정하는 농어가부업소득
    라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득
    마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양...
Chunk 3 (792글자): 라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금, 상병보상금(傷病補償金), 일시보상금, 장해보상금, 유족보상금, 행방불명보상금, ...

=== 검색된 문서 (6개) ===
문서 1: 제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 2...
---
문서 2: 2) 대학의 교직원 또는 대학과 고용관계가 있는 학생이 소속 대학에 설치된 「산업교육진흥 및 산학연협력촉진에 관한 법률」 제25조에 따른 산학협력단(이하 이 조에서 “산학협력단”이라 한다)으로부터 같은 법 제32조제1항제4호에 따라 받는 보상금
    저. 대통령령으로 정하는 복리후생적 성질의 급여
4. 연금소득 중 다음 각 목의 어느 하나에 해당하는 소득...
---
문서 3: 나. 「국가보안법」에 따라 받는 상금과 보로금
    다. 「상훈법」에 따른 훈장과 관련하여 받는 부상(副賞)이나 그 밖에 대통령령으로 정하는 상금과 부상
    라. 종업원등 또는 대학의 교직원이 퇴직한 후에 사용

In [ ]:
# MMR(Maximum Marginal Relevance) 검색 - 다양성 확보
retriever_mmr = vectorstore.as_retriever( # 텍스트 분할기 생성
    search_type="mmr",  # mmr 알고리즘 적용
    search_kwargs={
        "k": 6,         # 6개의 Document 객체를 반환하는데
        "fetch_k": 20   # 20개를 테스트로 넣어라
    }
)
retrieved_docs_mmr = retriever_mmr.invoke(query) # 텍스트 분할기 적용

context_mmr = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs_mmr)])

response_mmr = llm.invoke(f"""
{query}

법령 조항 (MMR 검색):
{context_mmr}

위 법령을 바탕으로 비과세소득 항목들을 체계적으로 정리해주세요.
""")

print("=== MMR 검색 결과 ===")
pprint(response_mmr.content)

=== MMR 검색 결과 ===
('소득세법 제12조에 따른 비과세소득 항목은 다음과 같이 체계적으로 정리할 수 있습니다.\n'
 '\n'
 '1. **공익신탁의 이익**\n'
 '   - 「공익신탁법」에 따른 공익신탁의 이익\n'
 '\n'
 '2. **사업소득**\n'
 '   - 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n'
 '   - 1개의 주택을 소유하는 자의 주택임대소득 (단, 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 '
 '제외)\n'
 '   - 대통령령으로 정하는 농어가부업소득\n'
 '   - 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n'
 '   - 조림기간 5년 이상인 임지의 임목의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액\n'
 '   - 대통령령으로 정하는 작물재배업에서 발생하는 소득\n'
 '   - 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n'
 '\n'
 '3. **근로소득과 퇴직소득**\n'
 '   - 대통령령으로 정하는 복무 중인 병이 받는 급여\n'
 '   - 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n'
 '   - 「산업재해보상보험법」에 따라 수급권자가 받는 각종 급여\n'
 '   - 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 각종 보상금 및 장의비\n'
 '\n'
 '4. **연금소득**\n'
 '   - 「국민연금법」, 「공무원연금법」 등 공적연금 관련법에 따라 받는 유족연금, 장애연금 등\n'
 '   - 「산업재해보상보험법」에 따라 받는 각종 연금\n'
 '   - 「국군포로의 송환 및 대우 등에 관한 법률」에 따른 국군포로가 받는 연금\n'
 '\n'
 '5. **기타소득**\n'
 '   - 「국가유공자 등 예우 및 지원에 관한 법률」에 따라 받는 보훈급여금 등\n'
 '   - 「국가보안법」에 따라 받는 상금과 보로금\n'
 '   - 「상훈법」에 따른 훈장과 관련